<a href="https://colab.research.google.com/github/advik-7/Automated-workflow-for-BPM-BPO/blob/main/Automated_Mail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DO not TOuch this code

In [2]:
!pip install python-dotenv

In [4]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [7]:
import openai
import imaplib
import smtplib
from email.message import EmailMessage
from email.header import decode_header
import email as email_lib
import os
from dotenv import load_dotenv
import re
import gradio as gr
from datetime import datetime, timedelta

# Load environment variables
load_dotenv(dotenv_path="pass.env")

# Email credentials
imap_host = "imap.gmail.com"
smtp_host = "smtp.gmail.com"
email_user = os.getenv("EMAIL_USER")
email_pass = os.getenv("EMAIL_PASS")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set OpenAI API key securely
openai.api_key = openai_api_key

def generate_reply(email_content):
    """
    Generate a reply using OpenAI's language model.
    """
    prompt = f"Reply to this email:\n\n{email_content}"
    response = openai.Completion.create(
        engine="text-davinci-003",  # Use a model that fits your requirements
        prompt=prompt,
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7,
    )
    reply = response.choices[0].text.strip()
    print("Generated reply:", reply)  # Debug print for generated reply
    return reply

# Function to fetch unread emails from the last 7 days
def fetch_unread_emails():
    try:
        print("Connecting to email server...")
        mail = imaplib.IMAP4_SSL(imap_host)
        mail.login(email_user, email_pass)
        mail.select("inbox")

        seven_days_ago = (datetime.now() - timedelta(days=7)).strftime("%d-%b-%Y")
        status, messages = mail.search(None, f'UNSEEN SINCE {seven_days_ago}')
        if status != 'OK':
            print("Error searching for emails:", status)
            return []

        email_ids = messages[0].split()
        print(f"Found {len(email_ids)} unread emails.")

        if not email_ids:
            return []

        emails = []
        for email_id in email_ids:
            status, msg_data = mail.fetch(email_id, "(RFC822)")
            if status != 'OK':
                print(f"Error fetching email {email_id}: {status}")
                continue

            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email_lib.message_from_bytes(response_part[1])
                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else "utf-8", errors="ignore")
                    else:
                        subject = subject or "No Subject"

                    from_ = msg.get("From")
                    if "<" in from_ and ">" in from_:
                        from_email = re.search(r"<(.+?)>", from_).group(1)
                    else:
                        from_email = from_

                    body = None
                    if msg.is_multipart():
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            if content_type == "text/plain":
                                body = part.get_payload(decode=True).decode(errors="ignore")
                                break
                    else:
                        body = msg.get_payload(decode=True).decode(errors="ignore")

                    body = body or "No content available"
                    emails.append({"subject": subject, "from": from_email, "body": body})

        mail.logout()
        return emails

    except Exception as e:
        print(f"Error fetching emails: {str(e)}")
        return []

# Function to send a reply
def send_reply(to_email, original_subject, reply_content):
    try:
        print(f"Sending reply to {to_email}...")
        with smtplib.SMTP_SSL(smtp_host, 465) as server:
            server.login(email_user, email_pass)
            msg = EmailMessage()
            msg["From"] = email_user
            msg["To"] = to_email
            msg["Subject"] = f"Re: {original_subject}"
            msg.set_content(reply_content)

            server.send_message(msg)
            print(f"Reply sent to {to_email}.")
    except Exception as e:
        print(f"Error sending reply to {to_email}: {str(e)}")

# Gradio interface function to show emails and generate replies
def email_interface():
    unread_emails = fetch_unread_emails()
    if not unread_emails:
        return gr.Interface(fn=lambda: "No unread emails found.", inputs=[], outputs="text", title="Email Auto-Reply System")

    email_options = [f"{email['subject']} from {email['from']}" for email in unread_emails]

    selected_email = gr.Dropdown(choices=email_options, label="Select an Email")

    def process_selected_email(selected):
        email_index = next(i for i, v in enumerate([(email["subject"], email["from"]) for email in unread_emails]) if f"{v[0]} from {v[1]}" == selected)
        email_to_reply = unread_emails[email_index]
        reply_content = generate_reply(email_to_reply["body"])
        send_reply(email_to_reply["from"], email_to_reply["subject"], reply_content)
        return reply_content

    interface = gr.Interface(fn=process_selected_email, inputs=selected_email, outputs="text", title="Email Auto-Reply System")
    return interface

if __name__ == "__main__":
    interface = email_interface()
    interface.launch()


Connecting to email server...
Found 1 unread emails.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f298bbc3f919f02a88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Use this Main functions

In [11]:
import gradio as gr
import smtplib
from email.message import EmailMessage
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path="pass.env")

# Email credentials
email_user = os.getenv("EMAIL_USER")
email_pass = os.getenv("EMAIL_PASS")
smtp_host = "smtp.gmail.com"

def send_email_with_attachments(to_email, subject, body_content, attachments=[]):
    """
    Function to send an email with the given subject, body content, and optional attachments.
    """
    try:
        # Create an email message
        msg = EmailMessage()
        msg["From"] = email_user
        msg["To"] = to_email
        msg["Subject"] = subject
        msg.set_content(body_content)

        # Add attachments if any
        for file_path in attachments:
            with open(file_path.name, "rb") as file:
                file_data = file.read()
                file_name = os.path.basename(file_path.name)
                msg.add_attachment(file_data, maintype="application", subtype="octet-stream", filename=file_name)

        # Send the email
        with smtplib.SMTP_SSL(smtp_host, 465) as server:
            server.login(email_user, email_pass)
            server.send_message(msg)
            return f"Email sent successfully to {to_email}."
    except Exception as e:
        return f"Error sending email: {str(e)}"

# Gradio interface
def gradio_email_interface(to_email, subject, body_content, attachments):
    return send_email_with_attachments(to_email, subject, body_content, attachments)

# Create Gradio interface
interface = gr.Interface(
    fn=gradio_email_interface,
    inputs=[
        gr.Textbox(label="Recipient Email", placeholder="Enter recipient's email"),
        gr.Textbox(label="Subject", placeholder="Enter email subject"),
        gr.Textbox(label="Message Content", placeholder="Type your email content here..."),
        gr.Files(label="Attachments (optional)")
    ],
    outputs="text",
    title="Email Sender",
    description="Send emails with optional attachments using Gradio."
)

# Launch the interface
if __name__ == "__main__":
    interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://adc91eb950d14017ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
